# Python機械学習クックブック

In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

## One-Hotエンコーディング

In [11]:
feature = np.array([["apple"],["banana"],["grape"],["apple"]])

# one-hotエンコーダを作成
one_hot = LabelBinarizer()

# 特徴量をワンホットエンコーディング
feature_one_hot = one_hot.fit_transform(feature)
print(feature_one_hot)

# ワンホットエンコードから逆変換
feature_inverse = one_hot.inverse_transform(feature_one_hot)
print(feature_inverse)

[[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]]
['apple' 'banana' 'grape' 'apple']


## カテゴリ特徴量の数値化

In [12]:
df = pd.DataFrame({"Score":["Low","Low","Medium","Medium","High"]})

# マップを作成
scale_mapper = {"Low":1,
"Medium":2,
"High":3,}

#マップを使って特徴量を置換
df["Score"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

## 不均衡データの取扱い

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

In [19]:
iris = load_iris()
feature = iris.data
target = iris.target

# 40データだけ抽出
feature = feature[40:,:]
target = target[40:]

# クラス0とそれ以外に分けて不均衡データを作成
target = np.where((target==0),0,1)

target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [22]:
# sklearnの標準機能で、不均衡データを打ち消すように訓練してくれる(方法1)

#　重みを作成
weights = {0: 0.9, 1: 0.1}

RandomForestClassifier(class_weight=weights)

# 学習と推論のコードはなかった

RandomForestClassifier(class_weight={0: 0.9, 1: 0.1})

In [23]:
# 多数派のクラスをダウンサンプリング（方法2）
i_class0 = np.where(target==0)[0]
i_class1 = np.where(target==1)[0]

n_class0 = len(i_class0)
n_class1 = len(i_class1)

# 多数派のクラスからランダムにデータを非復元抽出
i_class1_downsampled = np.random.choice(i_class1,size=n_class0,replace=False)

# クラス0とダウンサンプリングしたクラス1を結合
np.hstack((target[i_calass0],target[i_class1_downsampled]))

# 特徴量行列を結合
np.vstack((feature[i_class0,:],feature[i_class1_downsampled,:]))[0:5]

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4]])

In [24]:
# 少数派のクラスをアップサンプリング（方法3）
i_class0 = np.where(target==0)[0]
i_class1 = np.where(target==1)[0]

n_class0 = len(i_class0)
n_class1 = len(i_class1)

# 少数派のクラスからランダムにデータを復元抽出
i_class0_upsampled = np.random.choice(i_class0,size=n_class1,replace=True)

# アップサンプリングしたクラス0とクラス1を結合
np.concatenate((target[i_class0_upsampled],target[i_class1]))

# 特徴量行列を結合
np.vstack((feature[i_class0_upsampled,:],feature[i_class1,:]))[0:5]

array([[5. , 3.3, 1.4, 0.2],
       [4.5, 2.3, 1.3, 0.3],
       [5.1, 3.8, 1.9, 0.4],
       [4.5, 2.3, 1.3, 0.3],
       [5.3, 3.7, 1.5, 0.2]])